In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from shutil import copy2
import csv

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install tensorflow_hub

In [ ]:
import tensorflow_hub as hub

In [ ]:
tf.__version__

'2.8.0'

In [ ]:
import pandas as pd

# Increase precision of presented data for better side-by-side comparison
pd.set_option("display.precision", 8)

In [ ]:
data_root = ("/content/drive/MyDrive/EIS/Dataset")

In [ ]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)

datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="validation",
    shuffle=True,
    target_size=IMAGE_SHAPE
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE)

Found 293 images belonging to 5 classes.
Found 1180 images belonging to 5 classes.


In [ ]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((32, 224, 224, 3), (32, 5))

In [ ]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Earbuds': 0, 'Erasers': 1, 'Hand Bags': 2, 'Mats': 3, 'Rubicks Cubes': 4}


In [ ]:
!cat labels.txt

Earbuds
Erasers
Hand Bags
Mats
Rubicks Cubes

In [ ]:
IMAGE_SIZE = 224

In [ ]:
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/classification/5",
                 output_shape=[1280],
                 trainable=False),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              2555993   
                                                                 
 dropout (Dropout)           (None, 1001)              0         
                                                                 
 dense (Dense)               (None, 5)                 5010      
                                                                 
Total params: 2,561,003
Trainable params: 5,010
Non-trainable params: 2,555,993
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.compile(
  optimizer=optimizer,
  loss='categorical_crossentropy',
  metrics=['acc'])

In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit(
    train_generator,
    epochs=5,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

Epoch 1/5
 3/37 [=>............................] - ETA: 6:58 - loss: 3.4923 - acc: 0.1458

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


37/37 [==============================] - 520s 14s/step - loss: 1.0144 - acc: 0.7042 - val_loss: 0.2222 - val_acc: 0.9249
Epoch 2/5
37/37 [==============================] - 130s 3s/step - loss: 0.1327 - acc: 0.9568 - val_loss: 0.1977 - val_acc: 0.9317
Epoch 3/5
37/37 [==============================] - 129s 3s/step - loss: 0.0807 - acc: 0.9703 - val_loss: 0.1534 - val_acc: 0.9454
Epoch 4/5
37/37 [==============================] - 134s 4s/step - loss: 0.0565 - acc: 0.9814 - val_loss: 0.1417 - val_acc: 0.9556
Epoch 5/5
37/37 [==============================] - 129s 4s/step - loss: 0.0369 - acc: 0.9890 - val_loss: 0.1251 - val_acc: 0.9659


In [ ]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

10/10 [==============================] - 14s 1s/step - loss: 0.1251 - acc: 0.9659
Final loss: 0.13
Final accuracy: 96.59%


In [ ]:
model.save("/content/drive/MyDrive/EIS/Model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/EIS/Model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/EIS/Model/assets


In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/EIS/Model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

with open('/content/drive/MyDrive/EIS/Model/app.tflite','wb') as f:
  f.write(tflite_quant_model)

final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

10/10 [==============================] - 14s 1s/step - loss: 0.1251 - acc: 0.9659
Final loss: 0.13
Final accuracy: 96.59%
